# Extracting Premier League tables from wikipedia

I have also done this in R. Wanted to show methods using Python. 

In [116]:
import pandas as pd
import polars as pl
import numpy as np
import requests
import time
import io
import janitor

In [ ]:
# setup years and seasons 
years = np.arange(1992, 2025, 1)

years_sub = [str(y)[2:] for y in years + 1]

season = [f"{s}-{s + 1}" for s in years]

# vectors for setting up links 
fa_1 = np.arange(0, 7, 1)
nfa_1 = 7
fa_2 = np.arange(8, 15, 1)
nfa_2 = np.arange(15, len(years))

# construct the urls (list comp + list concat)
# similar to list = [list1] + [list2] + [list3] + [list4]
url_list = (
    [f"https://en.wikipedia.org/wiki/{years[i]}%E2%80%93{years_sub[i]}_FA_Premier_League" for i in fa_1] +
    [f"https://en.wikipedia.org/wiki/{years[nfa_1]}%E2%80%93{years_sub[nfa_1]}_Premier_League"] +
    [f"https://en.wikipedia.org/wiki/{years[i]}%E2%80%93{years_sub[i]}_FA_Premier_League" for i in fa_2] +
    [f"https://en.wikipedia.org/wiki/{years[i]}%E2%80%93{years_sub[i]}_Premier_League" for i in nfa_2]
)


In [102]:
# making custom bot to be nice to wikipedia
headers = {
    'User-Agent': 'MyBot/1.0 (andrewmoles2@gmail.com)'
}

# loop through links to pull all tables on each page
wiki_tables = []

for url in url_list:
    r = requests.get(url, headers=headers)
    time.sleep(1)
    df = pd.read_html(io.StringIO(r.text))
    wiki_tables.append(df)


In [103]:
# extract just the league tables - they all have 11 columns
# note you can do this manually but this is the better programmic way! 
league_tables = []

for tables in wiki_tables:
    filt = [len(c.columns) for c in tables]
    filt_index = [n for n, name in enumerate(filt) if name == 11]
    if filt_index:
        league_tables.append(tables[filt_index[0]])



In [ ]:
# add season as column for each data frame
for s in range(0, len(league_tables)):
    league_tables[s].insert(11, "season", season[s])



In [ ]:
# fix col names for 19-20
league_tables[27] = league_tables[27].rename(columns={'Teamvte': 'Team'})

Things left to do:

Clean each data frame in the list doing
  - Clean the names using janitor
  - remove the [] and other weird string issues
  - remove whitespace for teams
  - make sure pts is int
  - redo the gd column so it is not a string but an int. Previously easiest way was to just do a calculation on gf-ga

make a combined data frame and save
save each season from the list. Ideally this would be controlled so you create the file in python (checking if it exists) then write out each one. 